In [ ]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
df_RFM_histo_cleaned = dataiku.Dataset("df_RFM_histo_cleaned")
df = df_RFM_histo_cleaned.get_dataframe()

df["periode"] = pd.to_datetime(df["periode"].astype(str)+"-01")
df["semestre"] = df["periode"].dt.year.astype(str)+"-S"+((df["periode"].dt.month-1)//6+1).astype(str)
seuil_retention = 30
retention_df = df.groupby("semestre").agg(

    total_clients = ("client_id","nunique"),
    clients_actives = ('client_id', lambda x: ((df.loc[x.index, 'recency'] < seuil_retention) &
                                              (df.loc[x.index, 'periode'] == df.loc[x.index, 'periode'].max())).sum())

).reset_index()
retention_df['retention_rate'] = (retention_df['clients_actives'] / retention_df['total_clients']) * 100


# Write recipe outputs
df_tx_retention_globale = dataiku.Dataset("df_tx_retention_globale")
df_tx_retention_globale.write_with_schema(retention_df)
